In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

Read the Data into a dataframe

In [55]:
cfb = pd.read_csv('CFB2019.csv', index_col = 0)
cfb.head(2)

,Games,Win-Loss,Off Rank,Off Plays,Off Yards,Off Yards/Play,Off TDs,Off Yards per Game,Def Rank,Def Plays,...,Average Time of Possession per Game,Turnover Rank,Fumbles Recovered,Opponents Intercepted,Turnovers Gain,Fumbles Lost,Interceptions Thrown.y,Turnovers Lost,Turnover Margin,Avg Turnover Margin per Game
Team,,,,,,,,,,,,,,,,,,,,,
Air Force (Mountain West),13,11/2,51,881,5483,6.22,55,421.8,17,752,...,34:28:00,78,9,7,16,12,6,18,-2,-0.15
Akron (MAC),12,0-12,130,725,2918,4.02,14,243.2,83,871,...,26:29:00,128,6,5,11,12,14,26,-15,-1.25


In [56]:
#records were converted to dates, so they must be reversed to dates
def get_wins(date):
    wins = date.split('-')[0]
    wins = wins.split('/')[0]
    return wins
cfb['wins'] = cfb['Win-Loss'].apply(get_wins)

In [57]:
#remove unncessary columns from the dataframe
to_drop = ['Win-Loss']
for col in cfb.columns:
    if col[-4:] == 'Rank':
        to_drop.append(col)
cfb.drop(to_drop, axis = 1, inplace = True)

In [58]:
#calculate a team's win percentage, which will be the final response variable
cfb['win_percent'] = [np.NaN]*cfb.index.size
for team in cfb.index:
    cfb['win_percent'][team] = int(cfb['wins'][team])/int(cfb['Games'][team])

/Users/AngelaBeckham/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [70]:
#convert time of possion from a string in the form "min:sec" into a decimal
def convert_mins(time):
    mins = float(time.split(':')[0])
    secs = float(time.split(':')[1])
    secs = secs / 60
    return mins + secs
cfb['avg_top'] = cfb['Average Time of Possession per Game'].apply(convert_mins)
cfb['top'] = cfb['Time of Possession'].apply(convert_mins)
cfb.drop(['Average Time of Possession per Game', 'Time of Possession'], axis = 1, inplace = True)

In [68]:
cfb['avg_top'].value_counts()

29.316667    3
32.916667    2
30.050000    2
30.583333    2
29.333333    2
            ..
31.100000    1
27.216667    1
33.850000    1
30.600000    1
33.050000    1
Name: avg_top, Length: 118, dtype: int64

In [6]:
#scale turnover data to a per game basis since teams do not play the same number of games
cfb['opponents_int_game'] = cfb['Opponents Intercepted']/cfb['Games']
cfb['int_thrown_game'] = cfb['Interceptions Thrown.y']/cfb['Games']
cfb['fumbles_rec_game'] = cfb['Fumbles Recovered']/cfb['Games']
cfb['fumbles_lost_game'] = cfb['Fumbles Lost']/cfb['Games']

In [7]:
cfb.to_csv('cfb_improved.csv')